Cliff array determines truncation of Gaussian depending on position of 'x'.  This cutoff creates "cliffs" or sharp discontinuities.  Each Gaussian is multiplied by an indicator function: which creates smooth output in the active region, zero in the inactive region and discontinuous along the boundary.

In [ ]:
import numpy as np

class CliffGauss:
    def __init__(self, ctr=np.array([[0.5, 0.5]]), stddev=0.25, weight=1, cliff=1): 
        self.ctr = np.atleast_2d(ctr) 
        self.stddev = np.asarray(stddev) 
        self.weight = np.asarray(weight) 
        self.cliff = np.asarray(cliff) 
        self.d = self.ctr.shape[1] 
        
        if len(self.stddev) != len(self.weight) or len(self.stddev) != len(self.ctr): 
            raise ValueError("Incompatible arguments: stddev, weight, and ctr must have the same length") 
        
    def sample_data(self, x):
        x = np.atleast_2d(x)
        if x.shape[1] != self.d:
            raise ValueError("Dimensions not compatible")

        results = []
        for j in range(x.shape[0]):
            g = 0
            for i in range(self.ctr.shape[0]):
                ctr = self.ctr[i]
                r2 = np.sum((ctr - x[j]) ** 2)

                if self.cliff[i] == 0:
                    g += self.weight[i] * np.exp(-r2 / (self.stddev[i] ** 2))
                else:
                    coord = self.cliff[i]
                    dim = abs(coord) - 1  # convert to 0-based index
                    if dim >= self.d:
                        raise ValueError(f"Invalid cliff index {dim} for dimension {self.d}")
                
                    if (coord < 0 and (ctr - x[j])[dim] < 0) or (coord > 0 and (ctr - x[j])[dim] > 0):
                        g += self.weight[i] * np.exp(-r2 / (self.stddev[i] ** 2))

            results.append(g / self.ctr.shape[0])
        return np.array(results)





def rancliff(N=4, d=2, mnmn=0.5, mnsdev=0.2, sdevmn=1/4, sdevsdev=1/4, mnlim=(0,1), ncliff=0.5):
    ctr = np.zeros((N, d))
    for i in range(N):
        ctr[i] = np.random.normal(mnmn, mnsdev, d)
        while np.any((ctr[i] < mnlim[0]) | (ctr[i] > mnlim[1])):
            ctr[i] = np.random.normal(mnmn, mnsdev, d)

    sdev = np.abs(np.random.normal(sdevmn, sdevsdev, N))
    weight = np.full(N, 0.5)

    ncliff = int(ncliff * N)
    cliff = np.zeros(N, dtype=int)

    if ncliff > 0:
        # Use 0-based indexing, but encode direction using ±1
        dims = np.random.choice(d, ncliff, replace=True)  # values in 0 to d-1
        dirs = np.random.choice([-1, 1], ncliff)
        cliff[:ncliff] = (dims + 1) * dirs  # Store as ±(dim + 1)

    return CliffGauss(ctr, sdev, weight, cliff)